# BioPython

## TASK 1
### Own fasta parser

In [5]:
# 1
'''
Let's remeber how to use dictionaries.
Task: return a dictionary where 
    * keys are IDs of seqs from an input fasta file (prot.fasta),
    * key's values are seqs itself. 
'''

def my_own_fasta_parser(inFile):

    sequences = {}

    with open(inFile, "r") as file:
        
        for line in file:
            line = line.strip()
            if line[0] == ">":
                seq_id = line[1:]
            else:
                sequences[seq_id] = line

    return sequences

In [6]:
my_own_fasta_parser("prot.fasta")

{'seq0': 'FQTWEEFSRAAEKLYLADPMKVRVVLKYRHVDGNLCIKVTDDLVCLVYRTDQAQDVKKIEKF',
 'seq1': 'KYRTWEEFTRAAEKLYQADPMKVRVVLKYRHCDGNLCIKVTDDVVCLLYRTDQAQDVKKIEKFHSQLMRLME LKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq2': 'EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK',
 'seq3': 'MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK',
 'seq4': 'EEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVVSYEMRLFGVQKDNFALEHSLL',
 'seq5': 'SWEEFAKAAEVLYLEDPMKCRMCTKYRHVDHKLVVKLTDNHTVLKYVTDMAQDVKKIEKLTTLLMR',
 'seq6': 'FTNWEEFAKAAERLHSANPEKCRFVTKYNHTKGELVLKLTDDVVCLQYSTNQLQDVKKLEKLSSTLLRSI',
 'seq7': 'SWEEFVERSVQLFRGDPNATRYVMKYRHCEGKLVLKVTDDRECLKFKTDQAQDAKKMEKLNNIFF',
 'seq8': 'SWDEFVDRSVQLFRADPESTRYVMKYRHCDGKLVLKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq9': 'KNWEDFEIAAENMYMANPQNCRYTMKYVHSKGHILLKMSDNVKCVQYRAENMPDLKK',
 'seq10': 'FDSWDEFVSKSVELFRNHPDTTRYVVKYRHCEGKLVLKVTDNHECLKFKTDQAQDAKKMEK'}

## TASK 2
### Residue abundance

In [10]:
# 2
'''
Another super easy task.

We have the same fasta file (prot.fasta).
Now we want to get a list with the ids of protein seqs that have 
a relative frequency higher than a given threshold for a given residue.

And don't forget to use my_own_fasta_parser function from a previous task!
'''
import re

def my_own_residue_abundance(input_file, residue, threshold=0.2):
    
    seq_ids = []
    sequences = my_own_fasta_parser(input_file)

    for seq_id, sequence in sequences.items():
        freq = len(re.findall(residue, sequence)) / len(sequence)
        if freq > threshold:
            seq_ids.append(seq_id)

    return seq_ids

In [20]:
my_own_residue_abundance("prot.fasta", "K", 0.13)

['seq1', 'seq2', 'seq5', 'seq8', 'seq10']

*Check the relative abundance for each sequence*

In [31]:
from collections import Counter

sequences = my_own_fasta_parser("prot.fasta")
check = {}
for seq_id, seq in sequences.items():
    check[seq_id] = {key: value/len(seq) for key,value in Counter(seq).items()}["K"]
    print(seq_id, "-", check[seq_id])


seq0 - 0.11290322580645161
seq1 - 0.14018691588785046
seq2 - 0.13432835820895522
seq3 - 0.10344827586206896
seq4 - 0.08064516129032258
seq5 - 0.13636363636363635
seq6 - 0.11428571428571428
seq7 - 0.12307692307692308
seq8 - 0.1323529411764706
seq9 - 0.12280701754385964
seq10 - 0.14754098360655737


*Everything works fine*

## TASK 3
### Filtering for fasta files

In [26]:
# 3
'''
Let's practice in Bio package using.
Task:
1. read fasta file containing several DNA seqs (nucl.fasta)
2. subset seqs that have GC content > 45 and coding protein with aromaticity > 0.01
3. write a new fasta file with those protein(!) seqs
4. return the percentage of seqs that passed your filter
Hint: Bio.SeqIO, Bio.SeqRecord, Bio.SeqUtils
'''
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import GC
from Bio.SeqUtils.ProtParam import ProteinAnalysis

def my_own_filtering(input_file, output_file, filt_gc=45, filt_arom=0.01):
    
    sequences = {}
    c = 0
    
    with open(input_file, "r") as content:
        
        for record in SeqIO.parse(content, "fasta"):
            c+=1
            
            # calculate GC content using Bio
            calc_gc = GC(record.seq)
            
            # calculate aromaticity using Bio
            protein = str(record.seq.translate())
            x = ProteinAnalysis(protein)
            calc_arom = x.aromaticity()
            
            # so, now you can filter
            if calc_gc >= filt_gc and calc_arom >= filt_arom:
                sequences[record.id] = protein
    
    # write a new fasta file with aminoacids
    records = []
    for seq_id, seq in sequences.items():
        rec = SeqRecord(Seq(seq), id=seq_id, description="")
        records.append(rec)
    handle = open(output_file, "w")
    SeqIO.write(records, handle, "fasta")
    handle.close()

    # print the percentage
    perc = "%.2f" % (len(records) / c * 100)
    print(f"Percentage of seqs that passed the filter: [GC content > 45, aromaticity > 0.01]: {perc}%")

In [27]:
my_own_filtering("nucl.fasta", "filtered_nucl.fasta")

Percentage of seqs that passed the filter: [GC content > 45, aromaticity > 0.01]: 28.57%


## TASK 4
### Best alignment

In [3]:
# 4
"""
Continue practicing in Bio package using:
Task:
complete the following code that should be able to return 
the best alignment of two amino acid seqs (pairwise2.align.globalds)
based on BLOSUM62 matrix from Bio.SubsMat.MatrixInfo.
http://rosalind.info/glossary/blosum62/
"""

from Bio import pairwise2
from Bio.Align import substitution_matrices
from Bio.pairwise2 import format_alignment

def balign(first_seq, second_seq, gap_open=-10, gap_ext=-0.5):

    # Load the matrix
    matrix = substitution_matrices.load("BLOSUM62")

    # Generate the alignments
    alns = pairwise2.align.globalds(first_seq, second_seq, matrix,
                                    gap_open, gap_ext)

    # Extract the best alignment (first one in the alns list)
    top_aln = alns[0]

    # Print the alignment, ...
    aln_A, aln_B, score, begin, end = top_aln
    ans = format_alignment(aln_A, aln_B, score, begin, end)  
           
    return ans

In [5]:
al = balign('EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK',
            'MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK')
print(al)

EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK
 .||.||||.||.|||||||||||||||||||||||||.||||..|||||||||||||        |
-MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDV--------K
  Score=259.5



## TASK 5
### Translation

In [ ]:
# 5
""" You have some DNA sequence: AGTACTAGAGCATTCTATGGAG.
Find out which peptides could be created from it and sort them by their length.
Use as much Biopython modules as possible.
"""

In [15]:
from Bio.Seq import Seq

dna = Seq("AGTACTAGAGCATTCTATGGAG")

prots = []

# 1. Peptides from original dna string
## a) not taking stop codon into account
prots.append(str(dna.translate()))
## b) taking stop codon into acount
prots.append(str(dna.translate(to_stop = True)))


# 2. Peptides from second dna string (= reverse complement to original)
dna_rev_compl = dna.reverse_complement()
## a) not taking stop codon into account
prots.append(str(dna_rev_compl.translate()))
## b) taking stop codon into acount
prots.append(str(dna_rev_compl.translate(to_stop = True)))



# 3. from RNA 

## a) transcribe original dna and find start codon -> translate from start codon
rna = dna.transcribe()
rna_from_start = rna[rna.find("AUG"):]
# make this seq a multiple of three starting from start codon
rna_from_start = rna_from_start[: len(rna_from_start) - len(rna_from_start)%3]
prots.append(str(rna_from_start.translate()))

## b) transcribe reverse complement dna and find start codon -> translate from start codon
rna_rev = dna_rev_compl.transcribe()
rna_rev_from_start = rna_rev[rna_rev.find("AUG"):]
# make this seq a multiple of three starting from start codon
rna_rev_from_start = rna_rev_from_start[: len(rna_rev_from_start) - len(rna_rev_from_start)%3]
prots.append(str(rna_rev_from_start.translate()))


print(*sorted(prots, key=len), sep="\n")

ME
ML*Y
LHRML
STRAFYG
STRAFYG
LHRML*Y


## TASK 6

In [16]:
# 6
""" TASK: Try to create one-line function (without (!!!) using Bio package) 
that returns reverse complementary to a given sequence. 
Hint: using dictionaty & list comprehensions might be helpful.
"""

def rev_compl_one_line(seq):
    return "".join([{"A": "T", "T": "A", "G": "C", "C": "G"}[x] for x in seq])

In [18]:
rev_compl_one_line("ACTGAAATTT")

'TGACTTTAAA'